<h1 align="center"><font size="20"> GMP OpenAI Assitant </font></h1>



<center><img src="images/chroma.webp" width="80" height="50">   <img src="images/openai.png" width="50" height="50"</center>   <img src="images/wordmark-langchain.png" width="200" height="100"></center>

 <h1 align="center"><font size="4"> History Aware OpenAI Assistant with RAG-Chroma and LangChain</font></h1>

### Setup

**Note**: If working in colab run the commands below in a code line:

```
!pip install -q langchain
!pip install -q langchain-community
!pip install -U -q langchain-community pypdf
!pip install -q gradio
!pip install -q gradio-client
!pip install -q chromadb
!pip install -q langchain-chroma
!pip install -q langchain-openai
```

In [ ]:
# comment out if running in colab
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/chatbot/openai-rag/

/content/drive/MyDrive/chatbot/openai-rag


In [ ]:
import os
import getpass
import openai
import chromadb
import gradio as gr


from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())


if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")
else:
    openai.api_key  = os.environ['OPENAI_API_KEY']


**If runing in colab**

Comment lines 8-15 from the cell above and run:

```os.environ["OPENAI_API_KEY"] = "Your key" ```



In [ ]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import create_history_aware_retriever
from langchain.chains import (RetrievalQA,  ConversationalRetrievalChain,
                    LLMChain, StuffDocumentsChain)
from langchain_core.prompts import MessagesPlaceholder
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


### Paths to local database and RAG directory

In [ ]:
# Path to pdf files
file_path = "validation"
# Path to save vectorized documents
persist_directory='vectors'


In [ ]:
# Create the embeddings and models
models = ['o3-mini', 'gpt-4o' ]
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
# Note if using o3-mini remove the temperature parameter
llm = ChatOpenAI(model=models[1], temperature=0.05)

In [ ]:
# Create a prompt to instruct the model how to interact in a brief manner.
condense_template =(
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
    )


In [ ]:
# Create a prompt to instruct the model how to interact.
system_prompt = ("""
You are an AI compliance advisor system, and provide answers to questions by using fact based information when possible. You are well versed in Good Manufacturing Practices (GMP) and validation in life science industry. You can help provide insights about critical test for validation and draft validation and qualification protocols based on the context.
Use the following pieces of information to provide a concise answer to user questions {input}.
<context>
{context}
</context>
If you don't know the answer, just say that you don't know, don't try to make up an answer.
The response should be specific.
""")


In [ ]:
def rag_conversation(pdf_path, k, embeddings, model=llm, general_instruction=condense_template, system_instruction=system_prompt):

    # load documents pdf records from a directory
    loader = PyPDFDirectoryLoader(pdf_path)
    documents = loader.load()

    # split documents
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
    docs = text_splitter.split_documents(documents)
    chunked_documents = text_splitter.split_documents(docs)

    # create vector database from data
    client = chromadb.Client()

    if client.list_collections():
        collection = client.create_collection("validation")
    else:
        print("Collection already exists")
    vectordb = Chroma.from_documents(

        documents=chunked_documents,
        embedding=embeddings,
        persist_directory=persist_directory
    )

    # define retriever
    retriever = vectordb.as_retriever(search_type="similarity", search_kwargs={"k": k})

    condense_instruction = ChatPromptTemplate.from_messages(
    [
        ("system", general_instruction),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", "{input}"),
    ]
    )

    history_aware_retriever = create_history_aware_retriever(model, retriever,
                                                             condense_instruction)

    qa_prompt = ChatPromptTemplate.from_messages([
        ("system", system_instruction),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", "{input}"),
    ]
    )
    qa_chain = create_stuff_documents_chain(llm, qa_prompt)
    convo_qa_chain = create_retrieval_chain(history_aware_retriever, qa_chain)


    return convo_qa_chain

In [ ]:
# Instantiate the vector database
rag_chain = rag_conversation(file_path, 3, embeddings, llm, condense_template, system_prompt)

Collection already exists


In [ ]:
share = False
chat_history = []
with gr.Blocks(theme=gr.themes.Soft()) as demo:

    # Your app code here
    gr.Markdown(""" <h1><center>OpenAI GMP-Assistant</center></h1>
    <br>
    Ask your compliance assistant about GMP!""")
    chatbot = gr.Chatbot(type="messages", height=300)
    msg = gr.Textbox()

    with gr.Row():
        submit = gr.Button("Submit", variant="primary")
        clear = gr.ClearButton([msg, chatbot])

        def respond(input, chat_history):
            response = rag_chain.invoke({"input":input, "chat_history": chat_history})
            chat_history.append({"role": "user", "content": input})
            chat_history.append({"role": "assistant", "content": response['answer']})
            return "", chat_history

    submit.click(respond, [msg, chatbot], [msg, chatbot])

# Display Chatbot in URL running publicly for 72 hr
if share:
    demo.launch(share=share)

# Launches locally - debug to run in colab
else:
    demo.launch(debug=True)


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://ee9e60b9b6a402c0a2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://2d7a6044e2fd5ab1fa.gradio.live
Killing tunnel 127.0.0.1:7861 <> https://ee9e60b9b6a402c0a2.gradio.live
